In [0]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('price-procissing').getOrCreate()

In [0]:
%run /Workspace/apache-spark/databricks-project-fmcg-sports/utils/utilities

In [0]:
s3_bucket

In [0]:
# import file from s3 bucket
price_bronze = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load(f'{s3_bucket}/gross_price/gross_price.csv')
    .withColumn('readTimeStamp', f.current_timestamp())
)


In [0]:
price_bronze.summary().display()

In [0]:
#price_bronze.select('product_id').distinct().count()
#price_bronze.groupBy('product_id').count().show()
price_bronze.display()
# month column data is not consistanet, date is in different format gold table has year only. 
# gross_price has negative number as well unknown numbers. 

In [0]:
# fix the date column
# fix the gross price column if value is not number then chagne to 0 if it is negativ then change it positive it should be double. 
price_bronze.withColumns({
    'month': f.coalesce(
        f.try_to_date(f.col("month"), "yyyy-MM-dd"),
        f.try_to_date(f.col("month"), "dd/MM/yyyy"),
        f.try_to_date(f.col("month"), "yyyy/MM/dd"),
        f.try_to_date(f.col("month"), "dd-MM-yyyy")
    ),
    'gross_price_new': f.rlike(f.col("gross_price"),'^[0-9]+(\.[0-9]+)?$')
}).display()

In [0]:
price_bronze.withColumn(
    'gross_price_new',
      f.when(
          f.col("gross_price").rlike(r'^-?\d+(\.]d+)?$'),
          f.when(f.col("gross_price").cast('Double') < 0, f.col("gross_price") * -1)
          .otherwise(f.col("gross_price"))
      ).otherwise(0)
).display()